In [1]:
from generate_hnb import generate_hnb
from generate_ZI import generate_ZI
from AIC_BIC import calculate_aic_bic

k=3
n=300
y = generate_hnb(n=n, k=k, beta0=-1, beta1=1, gamma0=-2, gamma1=0.3, alpha0=0.5, alpha1=1, r=10, cov_type="nbinary")


In [2]:
X = y.loc[:, 'x'].values
y = y.loc[:, 'y'].values

In [3]:
import statsmodels.api as sm
import pandas as pd
X=sm.add_constant(X)
X=pd.DataFrame(X, columns=['intercept', 'x'])
y=pd.DataFrame(y, columns=['y'])

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    
# Add intercept column

# Prepare features and target
features = X.loc[:, ['intercept', 'x']]     
target = y['y']
    
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
        features, target, test_size=0.3, random_state=42
)

In [5]:
X_test.head()

,intercept,x
203,1.0,-0.783848
266,1.0,1.769998
152,1.0,-0.711741
9,1.0,-0.752898
233,1.0,0.322426


In [6]:
poi_k=2
nb_k=2 # if r unknown 2+1=3
zip_k=2+2
zinb_k=2+2+1 # if r known 2+2=4
zkip_k=2+2+2
zkinb_k=2+2+2+1 # if r known 2+2+2=6
zkhurdle_poi_k=2+2
zkicmp_k=2+2+2+1

In [7]:
X_train.shape[0]

210

In [8]:
import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import SET_USE_BIC_LLF
SET_USE_BIC_LLF(True)

poisson_model = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
poisson_pred = poisson_model.predict(X_test)
poisson_mse = mean_squared_error(y_test, poisson_pred)
poisson_mae = mean_absolute_error(y_test, poisson_pred)
poisson_r2 = r2_score(y_test, poisson_pred)  
poisson_llf = poisson_model.llf  # Log-likelihood at maximum
poisson_aic = poisson_model.aic  # AIC = -2*LL + 2*df_model
poisson_bic = poisson_model.bic  # BIC = -2*LL + log(n)*df_model
lol=calculate_aic_bic(n=X_train.shape[0], log_likelihood=poisson_llf, k=poi_k)
poisson_mse, poisson_mae, poisson_r2, poisson_llf, poisson_aic, poisson_bic, lol

(2.8093667667712685,
 1.3159724297759299,
 -0.0047185664200306565,
 -393.11787149813756,
 790.2357429962751,
 796.9299580577101,
 (790.2357429962751, 796.9299580577101))

In [9]:
nb2_model = sm.NegativeBinomial(y_train, X_train).fit()
nb_pred = nb2_model.predict(X_test)
nb_mse = mean_squared_error(y_test, nb_pred)
nb_mae = mean_absolute_error(y_test, nb_pred)
nb_r2 = r2_score(y_test, nb_pred)
# Get log-likelihood
nb_llf = nb2_model.llf  # Log-likelihood at maximum
nb_aic = nb2_model.aic  # AIC
nb_bic = nb2_model.bic  # BIC
lol=calculate_aic_bic(n=X_train.shape[0], log_likelihood=nb_llf, k=3)
nb_mse, nb_mae, nb_r2, nb_llf, lol, nb_aic, nb_bic

Optimization terminated successfully.
         Current function value: 1.707737
         Iterations: 8
         Function evaluations: 9
         Gradient evaluations: 9


(2.810297234549371,
 1.3160551043762758,
 -0.00505133117797274,
 -358.6246701101842,
 (723.2493402203684, 733.2906628125207),
 723.2493402203684,
 733.2906628125207)

In [10]:
from models.HurdlePoisson import ZeroKInflatedPoisson as ZKIHurdle

# ZeroKInflatedPoissonHurdle Model
model=ZKIHurdle(k=3)
res=model.fit(X_train, y_train)
ll=model.loglikelihood(X_train, y_train)
y_pred=model.predict_mean(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
lol=calculate_aic_bic(n=X_train.shape[0], log_likelihood=ll, k=zkhurdle_poi_k)
mse, mae, r2, ll, lol

(2.7821676030023643,
 1.2843317763338518,
 0.005008716308925187,
 -305.46902035442474,
 (618.9380407088495, 632.3264708317193))

In [11]:
from models.ZINB import ZINB_EM as ZINB_EM
from models.ZINB import predict_mean as ZINB_pred_mean

#ZINB
alpha=1/10
beta, gamma, fll = ZINB_EM(y_train.values, X_train.values, X_train.values, alpha) # alpha is a problem
mean_pred = ZINB_pred_mean(X_test.values, X_test.values, beta, gamma)
mse = mean_squared_error(y_test, mean_pred)
mae = mean_absolute_error(y_test, mean_pred)
r2 = r2_score(y_test, mean_pred)
lol=calculate_aic_bic(n=X_train.shape[0], log_likelihood=ll, k=zinb_k)
mse, mae, r2, ll, lol

(2.9854905459061802,
 1.2643257050193553,
 -0.06770601005960808,
 -305.46902035442474,
 (620.9380407088495, 637.6735783624368))

In [12]:
from models.ZIP import ZIP_EM as ZIP_EM
from models.ZIP import predict_mean as ZIP_pred_mean

#ZIP
beta, gamma, fll = ZIP_EM(y_train.values, X_train.values, X_train.values)
mean_pred = ZIP_pred_mean(X_test.values, X_test.values, beta, gamma)
mse = mean_squared_error(y_test, mean_pred)
mae = mean_absolute_error(y_test, mean_pred)
r2 = r2_score(y_test, mean_pred)
lol=calculate_aic_bic(n=X_train.shape[0], log_likelihood=fll, k=zip_k)
mse, mae, r2, fll, lol

(2.9760734112929437,
 1.266697392815185,
 -0.06433814435396035,
 -324.5271363651276,
 (657.0542727302552, 670.4427028531251))

In [13]:
from models.ZKINB import ZkINB_EM

model = ZkINB_EM()
res = model.fit_em(y_train, X_train.values, X_train.values, k)
y_pred_inb=model.predict(X_test.values, X_test.values)
mse = mean_squared_error(y_test, y_pred_inb)
mae = mean_absolute_error(y_test, y_pred_inb) 
r2 = r2_score(y_test, mean_pred)
lol=calculate_aic_bic(n=X_train.shape[0], log_likelihood=res['final_loglik'], k=zkinb_k)
mse, mae, r2, res['final_loglik'], lol

(2.992395222094953,
 1.26437731600578,
 -0.06433814435396035,
 -322.9048894704406,
 (659.8097789408812, 683.2395316559034))

In [14]:
from models.ZKIP import ZKIP_EM

model = ZKIP_EM(k_inflated=k)
res = model.fit(X_train.values, y_train.values)
y_pred_inb=model.predict_expected(X_test.values)
mse = mean_squared_error(y_test, y_pred_inb)
mae = mean_absolute_error(y_test, y_pred_inb) 
r2 = r2_score(y_test, mean_pred)
lol=calculate_aic_bic(n=X_train.shape[0], log_likelihood=res.final_loglik, k=zkinb_k)
mse, mae, r2, res.final_loglik, lol

(5.19131929234967,
 1.5615464298565191,
 -0.06433814435396035,
 -360.5340887405499,
 (735.0681774810998, 758.4979301961221))

In [15]:
from models.ZkICMP import ZkICMP

zkicmp_model = ZkICMP(k=k)
res = zkicmp_model.fit(X_train.values, y_train.values)
pred_results = zkicmp_model.predict(X_test.values)
_, _, zkicmp_predictions, _ = pred_results
zkicmp_mse = mean_squared_error(y_test, zkicmp_predictions)
zkicmp_mae = mean_absolute_error(y_test, zkicmp_predictions)
zkicmp_r2 = r2_score(y_test, zkicmp_predictions)
lol=calculate_aic_bic(n=X_train.shape[0], log_likelihood=-res.final_loglik, k=zkicmp_k)

zkicmp_mse, zkicmp_mae, zkicmp_r2, -res.final_loglik, lol

(5.197050586703708,
 1.5622435823112073,
 -0.8586299506512445,
 -360.53332913235056,
 (735.0666582647011, 758.4964109797234))